In [ ]:
# Init environment before running a demo notebook.
from resources.utils import *
init_demo()
from resources.utils import * # reload the global vars again

In [ ]:
# Create a test collection and stage a test item
collection = create_test_collection()
item = stage_test_item()

In [ ]:
import requests

openapi_endpoint = "/api" if local_mode else "/docs/openapi.json"

openapi = requests.get(f"{stac_client.href_catalog}{openapi_endpoint}", **apikey_headers).json()

ignore = [
    "/catalog/api",
    "/catalog/api.html",
    "/catalog/docs/oauth2-redirect",
    "/catalog/conformance",
    "/catalog/search",
    "/catalog/queryables",
    "/catalog/collections/{owner_id}:{collection_id}/queryables",
    "/catalog/collections/{collection_id}/queryables",
]

catalog_endpoints = [
    key
    for key, value in openapi["paths"].items()
    if key.startswith("/catalog") and ("get" in value)
]

if True:
    implicit_ownerid = []
    for endpoint in catalog_endpoints:
         implicit_ownerid.append(endpoint)
         if "{owner_id}:{collection_id}" in endpoint:
             implicit_ownerid.append(endpoint.replace("{owner_id}:{collection_id}", "{collection_id}"))
    catalog_endpoints = implicit_ownerid

for endpoint in catalog_endpoints:

    if endpoint in ignore:
        print(f"{endpoint}\n(ignored)\n")
        continue

    endpoint = endpoint.format(
        owner_id=stac_client.owner_id, collection_id=collection.id, item_id=item.id,
    )
    print(endpoint)

    stac = requests.get(f"{stac_client.href_catalog}{endpoint}", **apikey_headers).json()

    for field in "stac_extensions", "auth:schemes", "links", "assets":
        value = stac.get(field, None)
        if value:
            print(f"{field}: {json.dumps(value, indent=2)}")
    print()

    assert "https://stac-extensions.github.io/authentication/v1.1.0/schema.json" in stac["stac_extensions"]
    assert "auth:schemes" in stac

    for link in stac.get("links", []):
        assert link["auth:refs"] == ["apikey"]

    for asset in stac.get("assets", {}).values():
        assert asset["auth:refs"] == ["apikey"]